# Lesson 16 - Working with Columns

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

## Introduction

In this lesson, we will discuss a few tools for working with columns. In particular, we will explore how to select specific columns from a DataFrame, how to perform operations on columns, how to rename columns, and how to create new columns. We will use the gapminder dataset to illustrate these concepts.

In [0]:
gm_schema = (
    'country STRING, year INTEGER, continent STRING, population INTEGER, '
    'life_exp DOUBLE, gdp_per_cap DOUBLE, gini DOUBLE'
)

gm_df = (
  spark.read
  .option('delimiter', '\t')
  .option('header', True)
  .schema(gm_schema)
  .csv('/FileStore/tables/gapminder_data.txt')
)
    
gm_df.printSchema()

root
-- country: string (nullable = true)
-- year: integer (nullable = true)
-- continent: string (nullable = true)
-- population: integer (nullable = true)
-- life_exp: double (nullable = true)
-- gdp_per_cap: double (nullable = true)
-- gini: double (nullable = true)

We will draw a sample of rows to use in the examples presented in this notebook.

In [0]:
gm_sample = gm_df.sample(withReplacement=False, fraction=0.0003, seed=21)
gm_sample.show()
# gm_sample.show(truncate=False) --> will show the untruncated result

+--------------------+----+---------+----------+--------+-----------+----+
 country|year|continent|population|life_exp|gdp_per_cap|gini|
+--------------------+----+---------+----------+--------+-----------+----+
 Canada|1855| americas| 2880000| 41.3| 2300.0|30.7|
 Tonga|1910| asia| 22700| 28.3| 902.0|37.7|
 Russia|1919| europe| 77200000| 24.9| 1480.0|35.4|
 Vietnam|1922| asia| 18600000| 30.4| 1130.0|36.4|
 Trinidad and Tobago|1926| americas| 393000| 42.4| 2730.0|45.5|
Sao Tome and Prin...|1927| africa| 50900| 31.8| 1080.0|32.1|
 Congo, Dem. Rep.|1948| africa| 12000000| 41.2| 1510.0|42.5|
 Chad|1961| africa| 3060000| 44.9| 1470.0|25.6|
 Senegal|1982| africa| 5920000| 53.5| 1950.0|54.6|
 Egypt|2003| africa| 74000000| 69.9| 7640.0|32.2|
 Nigeria|2015| africa| 181000000| 64.0| 5670.0|43.0|
 Burundi|2017| africa| 10900000| 60.8| 705.0|38.6|
 Vanuatu|2018| asia| 282000| 64.3| 2900.0|37.6|
+--------------------+----+---------+----------+--------+-----------+----+

## The `select()` Transformation

We can create a DataFrame containing a subset of the columns from an existing DataFrme using the `select()` transformation. We indicate the columns that we would like to select by providing strings representing the names of the columns as arguments to `select()`.

In [0]:
gm_sample.select('country', 'year', 'population').show()

+--------------------+----+----------+
 country|year|population|
+--------------------+----+----------+
 Canada|1855| 2880000|
 Tonga|1910| 22700|
 Russia|1919| 77200000|
 Vietnam|1922| 18600000|
 Trinidad and Tobago|1926| 393000|
Sao Tome and Prin...|1927| 50900|
 Congo, Dem. Rep.|1948| 12000000|
 Chad|1961| 3060000|
 Senegal|1982| 5920000|
 Egypt|2003| 74000000|
 Nigeria|2015| 181000000|
 Burundi|2017| 10900000|
 Vanuatu|2018| 282000|
+--------------------+----+----------+

The `select()` transformation can also accept as an argument a list that contains the names of the columns that we wish to select. This allows us to use the `columns` attribute to easily selected some number of consecutive columns.

In [0]:
gm_sample.select(gm_df.columns[:4]).show(3)
gm_sample.select(gm_df.columns[4:]).show(3)

+-------+----+---------+----------+
country|year|continent|population|
+-------+----+---------+----------+
 Canada|1855| americas| 2880000|
 Tonga|1910| asia| 22700|
 Russia|1919| europe| 77200000|
+-------+----+---------+----------+
only showing top 3 rows

+--------+-----------+----+
life_exp|gdp_per_cap|gini|
+--------+-----------+----+
 41.3| 2300.0|30.7|
 28.3| 902.0|37.7|
 24.9| 1480.0|35.4|
+--------+-----------+----+
only showing top 3 rows

We can use the wildcard character `'*'` in `select()` to select every column within a DataFrame. This might not seem useful at the moment, but we will see later on that this does have practical applications.

In [0]:
gm_sample.select('*').show()

+--------------------+----+---------+----------+--------+-----------+----+
 country|year|continent|population|life_exp|gdp_per_cap|gini|
+--------------------+----+---------+----------+--------+-----------+----+
 Canada|1855| americas| 2880000| 41.3| 2300.0|30.7|
 Tonga|1910| asia| 22700| 28.3| 902.0|37.7|
 Russia|1919| europe| 77200000| 24.9| 1480.0|35.4|
 Vietnam|1922| asia| 18600000| 30.4| 1130.0|36.4|
 Trinidad and Tobago|1926| americas| 393000| 42.4| 2730.0|45.5|
Sao Tome and Prin...|1927| africa| 50900| 31.8| 1080.0|32.1|
 Congo, Dem. Rep.|1948| africa| 12000000| 41.2| 1510.0|42.5|
 Chad|1961| africa| 3060000| 44.9| 1470.0|25.6|
 Senegal|1982| africa| 5920000| 53.5| 1950.0|54.6|
 Egypt|2003| africa| 74000000| 69.9| 7640.0|32.2|
 Nigeria|2015| africa| 181000000| 64.0| 5670.0|43.0|
 Burundi|2017| africa| 10900000| 60.8| 705.0|38.6|
 Vanuatu|2018| asia| 282000| 64.3| 2900.0|37.6|
+--------------------+----+---------+----------+--------+-----------+----+

## Column Objects

The rows of DataFrames are represented by `Row` objects and the columns of a DataFrame are represented by `Column` objects. There are many situations when it is useful, or even necessary, to work directly with column objects. The `pyspark.sql.functions` module contains two functions name `col()` and `expr()` that can be used to create column objects. 

* The `col()` function accepts a string representing the name of a column within a DataFrame. 
* The `expr()` function accepts a string representing a SQL-like expression to be applied to a column within a DataFrame. 

The simplest type of expression that can be used within `expr()` is the name of a column. We will discuss more complicated expressions later.

In [0]:
from pyspark.sql.functions import col, expr

In [0]:
gini_col = col('gini')
pop_col = expr('population')

print(type(gini_col))
print(type(pop_col))

<class 'pyspark.sql.column.Column'>
<class 'pyspark.sql.column.Column'>

The arguments provided to `select()` can be strings representing column names, or they can be column objects associated with specific columns. This is illustrated in the cell below.

In [0]:
gm_sample.select('country', 'year', col('gini'), expr('population') ).show()

+--------------------+----+----+----------+
 country|year|gini|population|
+--------------------+----+----+----------+
 Canada|1855|30.7| 2880000|
 Tonga|1910|37.7| 22700|
 Russia|1919|35.4| 77200000|
 Vietnam|1922|36.4| 18600000|
 Trinidad and Tobago|1926|45.5| 393000|
Sao Tome and Prin...|1927|32.1| 50900|
 Congo, Dem. Rep.|1948|42.5| 12000000|
 Chad|1961|25.6| 3060000|
 Senegal|1982|54.6| 5920000|
 Egypt|2003|32.2| 74000000|
 Nigeria|2015|43.0| 181000000|
 Burundi|2017|38.6| 10900000|
 Vanuatu|2018|37.6| 282000|
+--------------------+----+----+----------+

## Column Operations

Numerical operations can be performed on column objects. The result is another column object. Additionally, we can indicate operations to be performed on the elements of a column within an expression string provided to `expr()`. Both of these approaches to performing column operations are illustrated below.

In [0]:
gini_col = col('gini') / 100
pop_col = expr('population / 1000000')

print(type(gini_col))
print(type(pop_col))

<class 'pyspark.sql.column.Column'>
<class 'pyspark.sql.column.Column'>

Note that column objects do not actually contain values from a column. In fact, there is nothing in our definition of the column objects above that even connect them to a specific DataFrame. Column objects contain a description of how the elements of a column should be calculated rather than containing the column values themselves. An operation represented by a column object is queued when it is supplied to the `select()` method, and is not actually performed until an action is called that requires the contents of that column.

In [0]:
gm_sample.select(
    'country', 'year', 
    col('gini') / 100, 
    expr('population / 1000000') 
).show()

+--------------------+----+------------+----------------------+
 country|year|(gini / 100)|(population / 1000000)|
+--------------------+----+------------+----------------------+
 Canada|1855| 0.307| 2.88|
 Tonga|1910| 0.377| 0.0227|
 Russia|1919| 0.354| 77.2|
 Vietnam|1922| 0.364| 18.6|
 Trinidad and Tobago|1926| 0.455| 0.393|
Sao Tome and Prin...|1927| 0.321| 0.0509|
 Congo, Dem. Rep.|1948| 0.425| 12.0|
 Chad|1961| 0.256| 3.06|
 Senegal|1982| 0.546| 5.92|
 Egypt|2003| 0.322| 74.0|
 Nigeria|2015| 0.43| 181.0|
 Burundi|2017| 0.386| 10.9|
 Vanuatu|2018| 0.376| 0.282|
+--------------------+----+------------+----------------------+

## Renaming Columns

Notice that Spark named the transformed columns in a way that describes the operation used to calculate their values. This results in somewhat unsightly column names. We can rename columns generated by `select()` in two ways:

* Every column object has an `alias()` method. This method accepts a string indicating the desired name for the column. 
* We can use the `AS` keywork within an expression string provided to `expr()` to set the name of the column being created. 

Both of these techniques are illustrated in the example below.

In [0]:
gm_sample.select(
    'country', 'year', 
    (col('gini') / 100).alias('gini_ratio'), 
    expr('population / 1000000 AS pop_in_mil') 
).show()

+--------------------+----+----------+----------+
 country|year|gini_ratio|pop_in_mil|
+--------------------+----+----------+----------+
 Canada|1855| 0.307| 2.88|
 Tonga|1910| 0.377| 0.0227|
 Russia|1919| 0.354| 77.2|
 Vietnam|1922| 0.364| 18.6|
 Trinidad and Tobago|1926| 0.455| 0.393|
Sao Tome and Prin...|1927| 0.321| 0.0509|
 Congo, Dem. Rep.|1948| 0.425| 12.0|
 Chad|1961| 0.256| 3.06|
 Senegal|1982| 0.546| 5.92|
 Egypt|2003| 0.322| 74.0|
 Nigeria|2015| 0.43| 181.0|
 Burundi|2017| 0.386| 10.9|
 Vanuatu|2018| 0.376| 0.282|
+--------------------+----+----------+----------+

## Operations Involving Multiple Columns

It is possible to perform calculations involving more than one column. This can be accomplished by performing operations directly on column objects representing the original columns, or by indicating the desired operation in an expression string provided to `expr()`. 

In the cell below, we will calculate the total GDP of each country in our sample by multiplying the per capita GDP by the population. For the sake of an example, we perform this calculation in two different ways.

In [0]:
gm_sample.select(
    '*',     
    (col('gdp_per_cap')*col('population')).alias('totalGDP1'),
    expr('gdp_per_cap * population AS totalGDP2')
).show()

+--------------------+----+---------+----------+--------+-----------+----+----------+----------+
 country|year|continent|population|life_exp|gdp_per_cap|gini| totalGDP1| totalGDP2|
+--------------------+----+---------+----------+--------+-----------+----+----------+----------+
 Canada|1855| americas| 2880000| 41.3| 2300.0|30.7| 6.624E9| 6.624E9|
 Tonga|1910| asia| 22700| 28.3| 902.0|37.7| 2.04754E7| 2.04754E7|
 Russia|1919| europe| 77200000| 24.9| 1480.0|35.4|1.14256E11|1.14256E11|
 Vietnam|1922| asia| 18600000| 30.4| 1130.0|36.4| 2.1018E10| 2.1018E10|
 Trinidad and Tobago|1926| americas| 393000| 42.4| 2730.0|45.5| 1.07289E9| 1.07289E9|
Sao Tome and Prin...|1927| africa| 50900| 31.8| 1080.0|32.1| 5.4972E7| 5.4972E7|
 Congo, Dem. Rep.|1948| africa| 12000000| 41.2| 1510.0|42.5| 1.812E10| 1.812E10|
 Chad|1961| africa| 3060000| 44.9| 1470.0|25.6| 4.4982E9| 4.4982E9|
 Senegal|1982| africa| 5920000| 53.5| 1950.0|54.6| 1.1544E10| 1.1544E10|
 Egypt|2003| africa| 74000000| 69.9| 7640.0|32.2| 5.6536E11| 5.6536E11|
 Nigeria|2015| africa| 181000000| 64.0| 5670.0|43.0|1.02627E12|1.02627E12|
 Burundi|2017| africa| 10900000| 60.8| 705.0|38.6| 7.6845E9| 7.6845E9|
 Vanuatu|2018| asia| 282000| 64.3| 2900.0|37.6| 8.178E8| 8.178E8|
+--------------------+----+---------+----------+--------+-----------+----+----------+----------+

## The `withColumn()` Transformation

We have seen that we can use `select()` to create new columns by performing operations on previously existing columns. Spark provides another transformation, named `withColumn()`, that can be used to add a column to a DataFrame. This method expects two arguments. The first argument should be a string representing the name of the new column. The second argument should be an expression that calculates the values for the new column from previously existing columns. Note that this method is a transformation that returns a DataFrame that includes the new column as well as all of the original columns.

In [0]:
gm_sample.withColumn('totalGDP', expr('gdp_per_cap * population') ).show()

+--------------------+----+---------+----------+--------+-----------+----+----------+
 country|year|continent|population|life_exp|gdp_per_cap|gini| totalGDP|
+--------------------+----+---------+----------+--------+-----------+----+----------+
 Canada|1855| americas| 2880000| 41.3| 2300.0|30.7| 6.624E9|
 Tonga|1910| asia| 22700| 28.3| 902.0|37.7| 2.04754E7|
 Russia|1919| europe| 77200000| 24.9| 1480.0|35.4|1.14256E11|
 Vietnam|1922| asia| 18600000| 30.4| 1130.0|36.4| 2.1018E10|
 Trinidad and Tobago|1926| americas| 393000| 42.4| 2730.0|45.5| 1.07289E9|
Sao Tome and Prin...|1927| africa| 50900| 31.8| 1080.0|32.1| 5.4972E7|
 Congo, Dem. Rep.|1948| africa| 12000000| 41.2| 1510.0|42.5| 1.812E10|
 Chad|1961| africa| 3060000| 44.9| 1470.0|25.6| 4.4982E9|
 Senegal|1982| africa| 5920000| 53.5| 1950.0|54.6| 1.1544E10|
 Egypt|2003| africa| 74000000| 69.9| 7640.0|32.2| 5.6536E11|
 Nigeria|2015| africa| 181000000| 64.0| 5670.0|43.0|1.02627E12|
 Burundi|2017| africa| 10900000| 60.8| 705.0|38.6| 7.6845E9|
 Vanuatu|2018| asia| 282000| 64.3| 2900.0|37.6| 8.178E8|
+--------------------+----+---------+----------+--------+-----------+----+----------+

It is perhaps a bit easier to use `withColumn()` if we are creating only a single column. But if we are creating multiple new columns, or want to also remove some of the previous columns, then `select()` is typically a better tool to use.